In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import json
import pickle

In [3]:
skill_program_df = pd.read_excel("TrainingProgram_Dataset.xlsx", sheet_name = 'Completion Data Clean', header=0)
userinfo_app_demo_df = pd.read_excel("TrainingProgram_Dataset.xlsx", sheet_name = 'App & Demo Data ', header=0)
essay_df = pd.read_excel("TrainingProgram_Dataset.xlsx", sheet_name = 'Essay Scores & Distance', header=0)

In [4]:
skill_program_df.columns = [c.lower().replace(' ','_').replace('_/_', '-').replace('(', '').replace(')', '').replace('_-_', '_') for c in skill_program_df.columns.tolist() ]
userinfo_app_demo_df.columns = [c.lower().replace(' ','_').replace('_/_', '-').replace('(', '').replace(')', '').replace('_-_', '_') for c in userinfo_app_demo_df.columns.tolist() ]
essay_df.columns = [c.lower().replace(' ','_').replace('_/_', '-').replace('(', '').replace(')', '').replace('_-_', '_') for c in essay_df.columns.tolist() ]

In [5]:
skill_program_df = skill_program_df.astype({'canvas_course_name':'category', 'unit':'category','record_type':'category','course_name':'category','section_name':'category'})
#skill_program_df = skill_program_df.astype({'canvas_course_name':'category', 'unit':'category','record_type':'category','course_name':'category','section_name':'category'})

In [5]:
skill_program_df.columns

Index(['canvas_user_id', 'course_id', 'canvas_course_name', 'day_of_start_at',
       'fake_applicant_id', 'fake_first_name-given_name', 'title',
       'title_clean', 'record_type', 'unit', 'include?', 'day_of_submitted_at',
       'day_of_due_at', 'grade', 'excused', 'submitted?', 'attendance',
       'section_canvas_course_id', 'course_name', 'section_name',
       'day_of_course_start_date', 'enrollment_status', 'date_difference',
       'completed?'],
      dtype='object')

In [6]:
userinfo_app_demo_df.columns

Index(['course_name', 'section_name', 'day_of_course_start_date',
       'day_of_applied_to_course_date', 'fake_applicant_id',
       'fake_first_name-given_name', 'fake_last_name-surname_',
       'fake_email_address', 'age_birthday_masked', 'race', 'gender', 'income',
       'education', 'major', 'city', 'state', 'zip_code', 'promise_zone?',
       'maxlearner_test_score', 'points_possible_on_learner_test',
       'primary_interest_in_course', 'hours_coded',
       'how_many_hours_a_week_can_you_commit_to_class', 'enrollment_status',
       'promise_zone_indicator', 'hacker_rank_score', 'essay_1_-_score_1',
       'essay_1_-_score_2', 'essay_1_-_score_3', 'essay_2_-_score_1',
       'essay_2_-_score_2', 'essay_2_-_score_3', 'essay_3_-_score_1',
       'essay_3_-_score_2', 'essay_3_-_score_3', 'max_essay_score',
       'average_essay_score', 'completed?'],
      dtype='object')

In [43]:
#skill_program_df.head(100000).astype({'canvas_course_name':'category', 'unit':'category','record_type':'category','course_name':'category','section_name':'category'}).memory_usage(deep=True)
#skill_program_df.head(100000).astype({'canvas_course_name':'category', 'unit':'category','record_type':'category','course_name':'category','section_name':'category'}).info(memory_usage='deep')
skill_program_df = skill_program_df.astype({'canvas_course_name':'category', 'unit':'category','record_type':'category','course_name':'category','section_name':'category'})

In [191]:
userinfo_app_demo_df.loc[userinfo_app_demo_df.average_essay_score.isnull()].shape

(1321, 38)

In [6]:
merged_df = pd.merge(skill_program_df, userinfo_app_demo_df, how='left', left_on=['fake_applicant_id', 'course_name','section_name'], right_on=['fake_applicant_id', 'course_name','section_name'])
#merged2_df = pd.merge(skill_program_df, userinfo_app_demo_df, how='outer', left_on=['fake_applicant_id'], right_on=['fake_applicant_id'])

In [193]:
merged_df.loc[merged_df.average_essay_score.isnull()].shape

(109782, 59)

In [200]:
merged2_df.loc[merged2_df.average_essay_score.notnull()].shape

(52770, 61)

In [10]:
merged_df.head(3)

,canvas_user_id,course_id,canvas_course_name,day_of_start_at,fake_applicant_id,fake_first_name-given_name_x,title,title_clean,record_type,unit,...,essay_1_-_score_3,essay_2_-_score_1,essay_2_-_score_2,essay_2_-_score_3,essay_3_-_score_1,essay_3_-_score_2,essay_3_-_score_3,max_essay_score,average_essay_score,completed?_y
0,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ˜ Blogz,Blogz,Large Assignment,Unit 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ˜ Build-a-Blog,Build-a-Blog,Large Assignment,Unit 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ¹ Chapter 10 Assignment,Chapter 10 Assignment,Small Assignment,Unit 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [16]:
merged_df.columns

Index(['canvas_user_id', 'course_id', 'canvas_course_name', 'day_of_start_at',
       'fake_applicant_id', 'fake_first_name-given_name_x', 'title',
       'title_clean', 'record_type', 'unit', 'include?', 'day_of_submitted_at',
       'day_of_due_at', 'grade', 'excused', 'submitted?', 'attendance',
       'section_canvas_course_id', 'course_name', 'section_name',
       'day_of_course_start_date_x', 'enrollment_status_x', 'date_difference',
       'completed?_x', 'day_of_course_start_date_y',
       'day_of_applied_to_course_date', 'fake_first_name-given_name_y',
       'fake_last_name-surname_', 'fake_email_address', 'age_birthday_masked',
       'race', 'gender', 'income', 'education', 'major', 'city', 'state',
       'zip_code', 'promise_zone?', 'maxlearner_test_score',
       'points_possible_on_learner_test', 'primary_interest_in_course',
       'hours_coded', 'how_many_hours_a_week_can_you_commit_to_class',
       'enrollment_status_y', 'promise_zone_indicator', 'hacker_rank_scor

In [152]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159080 entries, 0 to 159079
Data columns (total 59 columns):
canvas_user_id                                   159079 non-null float64
course_id                                        159079 non-null float64
canvas_course_name                               159079 non-null category
day_of_start_at                                  159079 non-null datetime64[ns]
fake_applicant_id                                159080 non-null int64
fake_first_name-given_name_x                     159080 non-null object
title                                            159079 non-null object
title_clean                                      159079 non-null object
record_type                                      157524 non-null category
unit                                             157524 non-null category
include?                                         159080 non-null object
day_of_submitted_at                              27611 non-null datetime64[ns]
day

In [120]:
merged_df.head(5)

,canvas_user_id,course_id,canvas_course_name,day_of_start_at,fake_applicant_id,fake_first_name-given_name_x,title,title_clean,record_type,unit,...,essay_1_-_score_3,essay_2_-_score_1,essay_2_-_score_2,essay_2_-_score_3,essay_3_-_score_1,essay_3_-_score_2,essay_3_-_score_3,max_essay_score,average_essay_score,completed?_y
0,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ˜ Blogz,Blogz,Large Assignment,Unit 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ˜ Build-a-Blog,Build-a-Blog,Large Assignment,Unit 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ¹ Chapter 10 Assignment,Chapter 10 Assignment,Small Assignment,Unit 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ¹ Chapter 11 Assignment,Chapter 11 Assignment,Small Assignment,Unit 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,1822.0,77.0,Org101 St. Louis - South County (Summer 2017),2017-07-17,10124,Anne Aranda,ðŸ¹ Chapter 12 Assignment,Chapter 12 Assignment,Small Assignment,Unit 1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [7]:
#essay_df = essay_df.apply(lambda x : x.fillna(0) if x.dtype.kind == 'biufc' else ' ')
#essay_df.loc[essay_df['max_score'] == 2].head(28)
cols_fillna = ['essay_1_score_1','essay_1_score_2','essay_1_score_3','essay_2_score_1','essay_2_score_2','essay_2_score_3','essay_3_score_1','essay_3_score_2','essay_3_score_3']
essay_df[cols_fillna]=essay_df[cols_fillna].fillna(-1)

In [8]:
def evaluateEssayAverage(row):
    score_list = [row.essay_1_score_1,row.essay_1_score_2,row.essay_1_score_3,row.essay_2_score_1,row.essay_2_score_2,row.essay_2_score_3,row.essay_3_score_1,row.essay_3_score_2,row.essay_3_score_3]
    percentage = (sum(x for x in score_list if x>=0)/(sum(1 for x in score_list if x>=0)*row.max_score + 0.001))*100
    return percentage

In [9]:
essay_df['essay_percentage'] = essay_df.apply(evaluateEssayAverage, axis=1)

In [123]:
essay_df.columns

Index(['course', 'fake_email', 'fake_helper', 'essay_1_score_1',
       'essay_1_score_2', 'essay_1_score_3', 'essay_2_score_1',
       'essay_2_score_2', 'essay_2_score_3', 'essay_3_score_1',
       'essay_3_score_2', 'essay_3_score_3', 'max_score', 'distance',
       'essay_average', 'essay_percentage'],
      dtype='object')

#### Merge the merged dataset with essay score dataset to get the correct averages

In [67]:
merged_total_df = pd.merge(merged_df, essay_df[['course','fake_email','essay_percentage']], how='left', left_on=['course_name','fake_email_address'], right_on=['course','fake_email'])

In [22]:
merged_total_df.columns

Index(['canvas_user_id', 'course_id', 'canvas_course_name', 'day_of_start_at',
       'fake_applicant_id', 'fake_first_name-given_name_x', 'title',
       'title_clean', 'record_type', 'unit', 'include?', 'day_of_submitted_at',
       'day_of_due_at', 'grade', 'excused', 'submitted?', 'attendance',
       'section_canvas_course_id', 'course_name', 'section_name',
       'day_of_course_start_date_x', 'enrollment_status_x', 'date_difference',
       'completed?_x', 'day_of_course_start_date_y',
       'day_of_applied_to_course_date', 'fake_first_name-given_name_y',
       'fake_last_name-surname_', 'fake_email_address', 'age_birthday_masked',
       'race', 'gender', 'income', 'education', 'major', 'city', 'state',
       'zip_code', 'promise_zone?', 'maxlearner_test_score',
       'points_possible_on_learner_test', 'primary_interest_in_course',
       'hours_coded', 'how_many_hours_a_week_can_you_commit_to_class',
       'enrollment_status_y', 'promise_zone_indicator', 'hacker_rank_scor

In [68]:
#merged_total_df['completed?_y'].value_counts()
merged_total_df['maxlearner_test_score_percent'] = merged_total_df['maxlearner_test_score']/merged_total_df['points_possible_on_learner_test']
merged_total_df.rename(columns={'completed?_y':'completed', 'enrollment_status_y':'enrollment_status'}, inplace=True)

In [69]:
merged_total_df = merged_total_df.dropna(subset=['maxlearner_test_score'])
merged_total_df = merged_total_df.dropna(subset=['record_type'])
merged_total_df = merged_total_df.dropna(subset=['education'])

In [70]:
drop_cols = ['day_of_start_at','title','day_of_submitted_at','day_of_due_at','grade','day_of_course_start_date_x','date_difference','day_of_course_start_date_y','day_of_applied_to_course_date','fake_first_name-given_name_y','fake_last_name-surname_','zip_code','essay_1_score_1','essay_1_score_2','essay_1_score_3','essay_2_score_1','essay_2_score_2','essay_2_score_3','essay_3_score_1','essay_3_score_2','essay_3_score_3','max_essay_score','fake_email','average_essay_score','course']
drop_cols2 = ['attendance','excused','submitted?','major','essay_percentage']
drop_cols3 = ['canvas_user_id','course_id','fake_first_name-given_name_x','section_canvas_course_id']
drop_cols4 = ['maxlearner_test_score','points_possible_on_learner_test']
drop_cols5 = ['promise_zone?','completed?_x','enrollment_status_x']
drop_cols6= ['course_name','section_name','fake_applicant_id','fake_email_address']

In [71]:
merged_total_df = merged_total_df.drop(drop_cols, axis=1)
merged_total_df = merged_total_df.drop(drop_cols2, axis=1)
merged_total_df = merged_total_df.drop(drop_cols3, axis=1)
merged_total_df = merged_total_df.drop(drop_cols4, axis=1)
merged_total_df = merged_total_df.drop(drop_cols5, axis=1)
###merged_total_df = merged_total_df.drop(drop_cols6, axis=1)

In [72]:
merged_total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 145362 entries, 0 to 146698
Data columns (total 24 columns):
canvas_course_name                               145362 non-null category
fake_applicant_id                                145362 non-null int64
title_clean                                      145362 non-null object
record_type                                      145362 non-null category
unit                                             145362 non-null category
include?                                         145362 non-null object
course_name                                      145362 non-null object
section_name                                     145362 non-null object
fake_email_address                               145362 non-null object
age_birthday_masked                              145362 non-null float64
race                                             145362 non-null object
gender                                           145362 non-null object
income             

In [62]:
##transform sentences to some Codes
code = {'Interested in seeking an apprenticeship/job in tech':'Apprentice_job','Interested in pursuing further education':'education' ,'General interest in coding':'coding','Professional development for my current job':'development'}
promise_zone_code = {'No':0, 'Promise Zone':1}
include_code = {'Yes':1, 'No':0}

In [73]:
merged_total_df['primary_interest_in_course'] = merged_total_df['primary_interest_in_course'].map(code)
merged_total_df['promise_zone_indicator'] = merged_total_df['promise_zone_indicator'].map(promise_zone_code)
merged_total_df['include?'] = merged_total_df['include?'].map(include_code)

In [390]:
merged_total_df['enrollment_status'].value_counts()

completed    83128
dropped      31055
enrolled     16479
withdrawn    14700
Name: enrollment_status, dtype: int64

In [64]:
merged_total_df.shape

(145362, 24)

In [39]:
import datetime
print(datetime.datetime.now())

2019-05-26 12:33:43.295273


In [74]:
print(datetime.datetime.now())
#merged_total_df.iloc[-round(merged_total_df.shape[0]*.3):].to_excel('ProcessedFile_30p.xlsx',header=True,index=False)
merged_total_df = merged_total_df.drop(drop_cols6, axis=1)
merged_total_interim_df = merged_total_df.drop('completed', axis=1)  ###Need completed for model building
merged_total_interim_df.to_csv('ProcessedTotalFile_Interim.csv',header=True,index=False)
print(datetime.datetime.now())

2019-05-26 14:04:20.787933
2019-05-26 14:04:24.156546


In [391]:
merged_total_df.head()

,canvas_course_name,title_clean,record_type,unit,include?,age_birthday_masked,race,gender,income,education,city,state,primary_interest_in_course,hours_coded,how_many_hours_a_week_can_you_commit_to_class,enrollment_status,promise_zone_indicator,hacker_rank_score,completed,maxlearner_test_score_percent
0,Org101 St. Louis - South County (Summer 2017),Blogz,Large Assignment,Unit 2,1,45.0,Caucasian or White,Male,"$40,000 - $50,000",Master's,Eureka,MO,Apprentice_job,6-50,21-40,withdrawn,0,9.333333,0.0,0.933333
1,Org101 St. Louis - South County (Summer 2017),Build-a-Blog,Large Assignment,Unit 2,1,45.0,Caucasian or White,Male,"$40,000 - $50,000",Master's,Eureka,MO,Apprentice_job,6-50,21-40,withdrawn,0,9.333333,0.0,0.933333
2,Org101 St. Louis - South County (Summer 2017),Chapter 10 Assignment,Small Assignment,Unit 1,1,45.0,Caucasian or White,Male,"$40,000 - $50,000",Master's,Eureka,MO,Apprentice_job,6-50,21-40,withdrawn,0,9.333333,0.0,0.933333
3,Org101 St. Louis - South County (Summer 2017),Chapter 11 Assignment,Small Assignment,Unit 1,1,45.0,Caucasian or White,Male,"$40,000 - $50,000",Master's,Eureka,MO,Apprentice_job,6-50,21-40,withdrawn,0,9.333333,0.0,0.933333
4,Org101 St. Louis - South County (Summer 2017),Chapter 12 Assignment,Small Assignment,Unit 1,1,45.0,Caucasian or White,Male,"$40,000 - $50,000",Master's,Eureka,MO,Apprentice_job,6-50,21-40,withdrawn,0,9.333333,0.0,0.933333


In [91]:
one_hot_cols = ['canvas_course_name','title_clean','record_type', 'unit','enrollment_status','race','gender','income','education', 'city', 'state','primary_interest_in_course', 'hours_coded','how_many_hours_a_week_can_you_commit_to_class']
one_hot_cols2 =['canvas_course_name','title_clean','record_type', 'unit','enrollment_status','race','gender','income','education', 'city', 'state','primary_interest_in_course', 'hours_coded','how_many_hours_a_week_can_you_commit_to_class']
print(len(one_hot_cols))
print(len(one_hot_cols2))

14
14


In [76]:
print(datetime.datetime.now())

merged_total_interim_load_df = pd.read_csv("ProcessedTotalFile_Interim.csv", header=0)
print(datetime.datetime.now())

2019-05-26 14:04:34.625003
2019-05-26 14:04:35.612183


In [78]:
merged_total_df.columns

Index(['canvas_course_name', 'title_clean', 'record_type', 'unit', 'include?',
       'age_birthday_masked', 'race', 'gender', 'income', 'education', 'city',
       'state', 'primary_interest_in_course', 'hours_coded',
       'how_many_hours_a_week_can_you_commit_to_class', 'enrollment_status',
       'promise_zone_indicator', 'hacker_rank_score', 'completed',
       'maxlearner_test_score_percent'],
      dtype='object')

In [92]:
##Continue with merged_total_df for model building...as this has the completed
merged_total_df_dummies = pd.get_dummies( merged_total_df, columns = one_hot_cols )

In [93]:
merged_total_df_dummies.shape

(145362, 545)

In [397]:
merged_total_df_dummies.head()

,age_birthday_masked,promise_zone_indicator,hacker_rank_score,completed,maxlearner_test_score_percent,canvas_course_name_C#/.NET SFL,canvas_course_name_C#/.NET St. Louis (Summer 2018),canvas_course_name_C#/.NET St. Louis (Winter 2018),canvas_course_name_C#/.NET St. Louis - Mentor Center (Winter 2017),canvas_course_name_C#/.NET St. Louis - North County (Summer 2017),...,primary_interest_in_course_education,hours_coded_0-5,hours_coded_250+,hours_coded_51-250,hours_coded_6-50,how_many_hours_a_week_can_you_commit_to_class_0-5,how_many_hours_a_week_can_you_commit_to_class_11-20,how_many_hours_a_week_can_you_commit_to_class_21-40,how_many_hours_a_week_can_you_commit_to_class_40+,how_many_hours_a_week_can_you_commit_to_class_6-10
0,45.0,0,9.333333,0.0,0.933333,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,45.0,0,9.333333,0.0,0.933333,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,45.0,0,9.333333,0.0,0.933333,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,45.0,0,9.333333,0.0,0.933333,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,45.0,0,9.333333,0.0,0.933333,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [94]:
###Split into X and Y
Y = merged_total_df_dummies['completed'].values
merged_total_df_dummies = merged_total_df_dummies.drop('completed', axis = 1)
X = merged_total_df_dummies.values

In [95]:
X.shape

(145362, 544)

In [399]:
merged_total_df_dummies.columns

Index(['age_birthday_masked', 'promise_zone_indicator', 'hacker_rank_score',
       'maxlearner_test_score_percent', 'canvas_course_name_C#/.NET SFL',
       'canvas_course_name_C#/.NET St. Louis (Summer 2018)',
       'canvas_course_name_C#/.NET St. Louis (Winter 2018)',
       'canvas_course_name_C#/.NET St. Louis - Mentor Center (Winter 2017)',
       'canvas_course_name_C#/.NET St. Louis - North County (Summer 2017)',
       'canvas_course_name_C#/.NET Tampa Bay (Fall 2017)',
       ...
       'primary_interest_in_course_education', 'hours_coded_0-5',
       'hours_coded_250+', 'hours_coded_51-250', 'hours_coded_6-50',
       'how_many_hours_a_week_can_you_commit_to_class_0-5',
       'how_many_hours_a_week_can_you_commit_to_class_11-20',
       'how_many_hours_a_week_can_you_commit_to_class_21-40',
       'how_many_hours_a_week_can_you_commit_to_class_40+',
       'how_many_hours_a_week_can_you_commit_to_class_6-10'],
      dtype='object', length=545)

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3)

In [97]:
clf=RandomForestClassifier(n_estimators=100)

In [98]:
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

In [99]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 1.0


In [102]:
##Save mode las pickle file
with open('RFModel_Hackathon2'+'.pickle', 'wb') as handle:
        pickle.dump(clf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [86]:
np.unique(y_test,return_counts=True)

(array([0., 1.]), array([18737, 24872], dtype=int64))

In [120]:
feature_imp = pd.Series(clf.feature_importances_,index=merged_total_df_dummies.columns).sort_values(ascending=False)
feature_imp.iloc[:10]

enrollment_status_completed                  0.463837
enrollment_status_dropped                    0.146846
enrollment_status_enrolled                   0.086732
enrollment_status_withdrawn                  0.066291
age_birthday_masked                          0.016904
unit_Unit 3                                  0.014153
maxlearner_test_score_percent                0.011071
hacker_rank_score                            0.011059
hours_coded_0-5                              0.006003
primary_interest_in_course_Apprentice_job    0.003671
dtype: float64

In [410]:
from sklearn.linear_model import LogisticRegression
logreg_red = LogisticRegression()
logreg_red.fit(X_train,y_train)
y_pred_class = logreg_red.predict(X_test)
#probs = logreg_red.predict_proba(X_test_std)
print(metrics.accuracy_score(y_test, y_pred_class))

C:\Users\jinu.jayan\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


1.0


In [49]:
with open('input.json') as json_file:  
    jsondata = json.load(json_file)

In [50]:
infer_df = pd.DataFrame(jsondata['students'], index=list(range(len(jsondata['students']))))

In [52]:
output_df = infer_df[['course_name','section_name','fake_applicant_id','fake_email_address']].copy()
drop_cols6= ['course_name','section_name','fake_applicant_id','fake_email_address']
infer_df = infer_df.drop(drop_cols6, axis=1)

In [105]:
output_df.shape

(2, 4)

In [103]:
Interim_load_df = pd.read_csv("ProcessedTotalFile_Interim.csv", header=0)
dataset_all = pd.concat(objs=[Interim_load_df, infer_df], axis=0,sort=False)
one_hot_cols2 = ['canvas_course_name','title_clean','record_type', 'unit','enrollment_status','race','gender','income','education', 'city', 'state','primary_interest_in_course', 'hours_coded','how_many_hours_a_week_can_you_commit_to_class']
dataset_all_dummies = pd.get_dummies( dataset_all, columns = one_hot_cols2 )
print(f"The OHE shape is {dataset_all_dummies.shape}")

The OHE shape is (145364, 544)


In [55]:
dataset_all_dummies.iloc[-2:]

,include?,age_birthday_masked,promise_zone_indicator,hacker_rank_score,completed,maxlearner_test_score_percent,canvas_course_name_C#/.NET SFL,canvas_course_name_C#/.NET St. Louis (Summer 2018),canvas_course_name_C#/.NET St. Louis (Winter 2018),canvas_course_name_C#/.NET St. Louis - Mentor Center (Winter 2017),...,primary_interest_in_course_education,hours_coded_0-5,hours_coded_250+,hours_coded_51-250,hours_coded_6-50,how_many_hours_a_week_can_you_commit_to_class_0-5,how_many_hours_a_week_can_you_commit_to_class_11-20,how_many_hours_a_week_can_you_commit_to_class_21-40,how_many_hours_a_week_can_you_commit_to_class_40+,how_many_hours_a_week_can_you_commit_to_class_6-10
0,0,22,0,6,NaN,0.6,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
1,1,22,0,6,NaN,0.6,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [107]:
X = dataset_all_dummies.values
savedModel = 'RFModel_Hackathon2.pickle'

In [108]:
with open(savedModel, 'rb') as handle:
        rf_clf_loaded = pickle.load(handle)
        print("model loaded.....")
y_pred_all =rf_clf_loaded.predict(X)
#output_df['Predition'] = y_pred


model loaded.....


In [111]:
output_df.shape[0]

2

In [113]:
y_pred_all[-output_df.shape[0]:]

array([0., 0.])

In [114]:
output_df['Predition'] = y_pred_all[-output_df.shape[0]:]

In [116]:
output_df.head()

,course_name,section_name,fake_applicant_id,fake_email_address,Predition
0,Org101 STL (July 2017),Org101 LaunchCode Mentor Center,78806,LisaVWebb@fleckens.hu,0.0
1,Org101 STL (July 2017),Org101 LaunchCode Mentor Center,78806,LisaVWebb@fleckens.hu,0.0


In [151]:
jsn = output_df.to_json(orient='records')
print(type(jsn))
print(jsn)
feature_imp = pd.DataFrame({'FeatureName' : dataset_all_dummies.columns,
                            'FeatureImportance' : rf_clf_loaded.feature_importances_}).sort_values(by = 'FeatureImportance',ascending=False)
#dict_features = feature_imp.iloc[:10].to_dict('records')
feature_json =feature_imp.iloc[:10].to_json(orient='records')
print(feature_json)
print(json.loads(feature_json))
#jsnobj.append(dict_features)
print("Show the final JSON obj...")
full_out = {'students':jsn, 'features':feature_json}
print("------------------------------------------------")
print(full_out)
print("------------------------------------------------")
print(json.dumps(full_out).replace('\\"',"\"").replace('"[', '[').replace(']"', ']'))

<class 'str'>
[{"course_name":"Org101 STL (July 2017)","section_name":"Org101 LaunchCode Mentor Center","fake_applicant_id":"78806","fake_email_address":"LisaVWebb@fleckens.hu","Predition":0.0},{"course_name":"Org101 STL (July 2017)","section_name":"Org101 LaunchCode Mentor Center","fake_applicant_id":"78806","fake_email_address":"LisaVWebb@fleckens.hu","Predition":0.0}]
[{"FeatureName":"enrollment_status_completed","FeatureImportance":0.4638365778},{"FeatureName":"enrollment_status_dropped","FeatureImportance":0.1468459078},{"FeatureName":"enrollment_status_enrolled","FeatureImportance":0.0867316505},{"FeatureName":"enrollment_status_withdrawn","FeatureImportance":0.066291389},{"FeatureName":"age_birthday_masked","FeatureImportance":0.016904466},{"FeatureName":"unit_Unit 3","FeatureImportance":0.0141531014},{"FeatureName":"maxlearner_test_score_percent","FeatureImportance":0.011070662},{"FeatureName":"hacker_rank_score","FeatureImportance":0.0110587915},{"FeatureName":"hours_coded_0-5

In [127]:
jsnobj[2]

[{'FeatureName': 'enrollment_status_completed',
  'FeatureImportance': 0.46383657779303455},
 {'FeatureName': 'enrollment_status_dropped',
  'FeatureImportance': 0.14684590777338205},
 {'FeatureName': 'enrollment_status_enrolled',
  'FeatureImportance': 0.08673165046049769},
 {'FeatureName': 'enrollment_status_withdrawn',
  'FeatureImportance': 0.06629138900627059},
 {'FeatureName': 'age_birthday_masked',
  'FeatureImportance': 0.016904465996773345},
 {'FeatureName': 'unit_Unit 3', 'FeatureImportance': 0.014153101373922252},
 {'FeatureName': 'maxlearner_test_score_percent',
  'FeatureImportance': 0.011070662023178074},
 {'FeatureName': 'hacker_rank_score',
  'FeatureImportance': 0.011058791537452142},
 {'FeatureName': 'hours_coded_0-5', 'FeatureImportance': 0.006002609502396392},
 {'FeatureName': 'primary_interest_in_course_Apprentice_job',
  'FeatureImportance': 0.003670865781864654}]

### 